# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv in Part I into a DataFrame
weather_summary_df = pd.read_csv("output_data/weather_summary.csv")
weather_summary_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Laguna,1,US,1591111708,60,38.42,-121.42,73.00,5.48
1,1,Adrar,96,MR,1591111709,15,20.50,-10.07,103.50,2.04
2,2,La Ronge,75,CA,1591111593,51,55.10,-105.28,59.00,13.87
3,3,Mahébourg,75,MU,1591111711,94,-20.41,57.70,71.60,12.75
4,4,Busselton,0,AU,1591111713,71,-33.65,115.33,52.00,6.11
...,...,...,...,...,...,...,...,...,...,...
555,555,Lukulu,0,ZM,1591112393,27,-14.37,23.24,70.25,4.29
556,556,Valparaíso,100,CL,1591112331,62,-33.04,-71.63,55.40,2.24
557,557,Marawi,97,PH,1591112396,91,8.00,124.29,75.00,5.01
558,558,Ouahigouya,77,BF,1591112397,17,13.58,-2.42,105.42,6.85


In [3]:
# Drop Null Values
clean_weather_data = weather_summary_df.dropna()
del clean_weather_data["Unnamed: 0"]
clean_weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Laguna,1,US,1591111708,60,38.42,-121.42,73.00,5.48
1,Adrar,96,MR,1591111709,15,20.50,-10.07,103.50,2.04
2,La Ronge,75,CA,1591111593,51,55.10,-105.28,59.00,13.87
3,Mahébourg,75,MU,1591111711,94,-20.41,57.70,71.60,12.75
4,Busselton,0,AU,1591111713,71,-33.65,115.33,52.00,6.11
...,...,...,...,...,...,...,...,...,...
555,Lukulu,0,ZM,1591112393,27,-14.37,23.24,70.25,4.29
556,Valparaíso,100,CL,1591112331,62,-33.04,-71.63,55.40,2.24
557,Marawi,97,PH,1591112396,91,8.00,124.29,75.00,5.01
558,Ouahigouya,77,BF,1591112397,17,13.58,-2.42,105.42,6.85


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

#Use the Lat and Lng as locations and Humidity as the weight.

locations = clean_weather_data[["Lat", "Lng"]]

humidity = clean_weather_data["Humidity"].astype(float)

In [5]:
# Add Heatmap layer to map.
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the cities to fit weather conditions

# weather conditions with zero cloudiness
no_cloudiness = clean_weather_data[clean_weather_data["Cloudiness"] == 0]

# weather conditions with max temp above 70 degrees F
above_70_degrees = no_cloudiness[no_cloudiness["Max Temp"] > 70]

# weather conditions with max temp below 80 degrees F
below_80_degrees = above_70_degrees[above_70_degrees["Max Temp"] < 80]

# weather condtions with wind speed below 10 mph
low_windspeed = below_80_degrees[below_80_degrees["Wind Speed"] < 10]

# weather conditions with ideal humidity below 60 %
ideal_weather = low_windspeed[low_windspeed["Humidity"] < 60]

ideal_weather

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
67,Camargo,0,MX,1591111792,41,27.67,-105.17,79.18,9.69
107,Carnarvon,0,AU,1591111841,59,-24.87,113.63,71.65,5.82
165,Nur-Sultan,0,KZ,1591111749,41,51.18,71.45,75.20,4.47
426,Divinópolis,0,BR,1591112236,57,-20.14,-44.88,74.23,6.02
531,Yinchuan,0,CN,1591112364,15,38.47,106.27,75.63,3.47
555,Lukulu,0,ZM,1591112393,27,-14.37,23.24,70.25,4.29


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_weather[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
67,Camargo,MX,27.67,-105.17,
107,Carnarvon,AU,-24.87,113.63,
165,Nur-Sultan,KZ,51.18,71.45,
426,Divinópolis,BR,-20.14,-44.88,
531,Yinchuan,CN,38.47,106.27,


In [8]:

# Loop through dataframe to get closest hotel
for index, row  in hotel_df.iterrows():
    lat = row ["Lat"]
    lng = row ["Lng"]
    
    target_type = "lodging"
    coords = f"{lat},{lng}"
    radius = 5000


    params = {
        "location":  coords, 
        "types": target_type,
        "radius": radius,
        "key": g_key}

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 
    
    hotel_request = requests.get(base_url, params)
    hotel_response = hotel_request.json()

    try:
        #print(hotel_response["results"][0]["name"])
        hotel_df.loc[index, "Hotel Name"] = hotel_response["results"][0]["name"]
        
    except:
        print(f"Nearest hotel not found!")  
        


        


In [9]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
67,Camargo,MX,27.67,-105.17,Hotel Santa Fe
107,Carnarvon,AU,-24.87,113.63,Hospitality Carnarvon
165,Nur-Sultan,KZ,51.18,71.45,King Hotel Astana
426,Divinópolis,BR,-20.14,-44.88,Hotel Center Palace
531,Yinchuan,CN,38.47,106.27,JJ Inns - Yinchuan New Moon Square Branch
555,Lukulu,ZM,-14.37,23.24,Ruth Guest House


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))